# document count update


- Name: **Shengqian Wang**
- Github username: **Objqian**

In [2]:
import pandas as pd
#import geopandas as gpd
from matplotlib import pyplot as plt
import numpy as np
pd.options.display.max_columns = 999
#import hvplot.pandas
import folium
import nltk
import string
#import carto2gpd
#from datetime import datetime
#from datetime import date
#from sklearn.cluster import dbscan 
np.random.seed(42)
import json
import requests
import geopandas as gpd

from shapely.geometry import MultiPolygon, Polygon

from shapely import wkt


Import **try** - csv and  **two** shp

In [2]:
try10 = pd.read_csv('E:/Downloads/test2.csv')
#view try0 in a new window
try10 = try10[4:12]
try10

two = gpd.read_file(r'D:\OneDrive - PennO365\penn\5th\MUSA611\world_exp.shp')
# rebane To_Country to Country
two = two.rename(columns={'To_Country': 'Country'})
# join the geodataframe with the cleaned up csv dataframe
#two = try10.join(two.set_index('Country'), on='Country') #我觉得也可以等到最后一起join
two

,Country,Sex,Sta,2022,2023
4,Denmark,Men,Immigrations,0,3
5,Denmark,Men,Emigrations,0,3
6,Denmark,Women,Immigrations,0,3
7,Denmark,Women,Emigrations,0,3
8,Faroe Islands,Men,Immigrations,1,4
9,Faroe Islands,Men,Emigrations,0,3
10,Faroe Islands,Women,Immigrations,0,3
11,Faroe Islands,Women,Emigrations,0,3


### pivot

In [30]:
try0 = pd.read_csv('E:/Downloads/test1.csv')
#view try0 in a new window
try0 = try0[4:12]
try0

tr30 = try0.pivot(index='Country', columns=['Sex','Sta'], values=['2022'])
tr30 = tr30.reset_index()
#tr30 = tr30[0:2]
tr30

Country         2022                                     
Sex                         Men                    Women            
Sta                Immigrations Emigrations Immigrations Emigrations
0          Denmark            0           0            0           0
1    Faroe Islands            1           0            0           0

## Combine column name with/ with out year

In [31]:
# with year
#tr30.columns = [' '.join(col).strip() for col in tr30.columns.values]
# without year
tr30.columns = [' '.join(col).strip() if '2022' not in col else ' '.join(col[1:]).strip() for col in tr30.columns.values]
tr30


,Country,Men Immigrations,Men Emigrations,Women Immigrations,Women Emigrations
0,Denmark,0,0,0,0
1,Faroe Islands,1,0,0,0


In [33]:
#tr30.reset_index(inplace=True)
each = tr30.join(two.set_index('Country'), on='Country') 
each

,Country,Men Immigrations,Men Emigrations,Women Immigrations,Women Emigrations,continent,area_km2,pop,lifeExp,gdpPercap,geometry
0,Denmark,0,0,0,0,Denmark,42759.927703,5643475.0,80.70,45057.07417,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8..."
1,Faroe Islands,1,0,0,0,Denmark,1393.000000,54738.0,83.09,58585.00000,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61..."


In [81]:
#each.to_json()

TypeError: keys must be str, int, float, bool or None, not tuple

## Second Method - got a GeoDF with a children?

In [3]:
IM_csv = pd.read_csv('E:/Downloads/test2.csv')
#view try0 in a new window
IM_csv = IM_csv[4:12]
IM_csv

,Country,Sex,Sta,2022,2023
4,Denmark,Men,Immigrations,0,3
5,Denmark,Men,Emigrations,0,3
6,Denmark,Women,Immigrations,0,3
7,Denmark,Women,Emigrations,0,3
8,Faroe Islands,Men,Immigrations,1,4
9,Faroe Islands,Men,Emigrations,0,3
10,Faroe Islands,Women,Immigrations,0,3
11,Faroe Islands,Women,Emigrations,0,3


In [4]:
ct_shp = gpd.read_file(r'D:\OneDrive - PennO365\penn\5th\MUSA611\world_exp.shp')
# rebane To_Country to Country
ct_shp = ct_shp.rename(columns={'To_Country': 'Country'})
# join the geodataframe with the cleaned up csv dataframe
ct_shpf  = ct_shp.merge(IM_csv, on='Country', how='left')
# ct_shpf = IM_csv.join(ct_shp.set_index('Country'), on='Country')
ct_shpf2 = ct_shpf[['Country','Sex','Sta','2022','2023','geometry']]
ct_shpf

,Country,continent,area_km2,pop,lifeExp,gdpPercap,geometry,Sex,Sta,2022,2023
0,Denmark,Denmark,42759.927703,5643475.0,80.70,45057.07417,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",Men,Immigrations,0,3
1,Denmark,Denmark,42759.927703,5643475.0,80.70,45057.07417,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",Men,Emigrations,0,3
2,Denmark,Denmark,42759.927703,5643475.0,80.70,45057.07417,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",Women,Immigrations,0,3
3,Denmark,Denmark,42759.927703,5643475.0,80.70,45057.07417,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",Women,Emigrations,0,3
4,Faroe Islands,Denmark,1393.000000,54738.0,83.09,58585.00000,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",Men,Immigrations,1,4
5,Faroe Islands,Denmark,1393.000000,54738.0,83.09,58585.00000,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",Men,Emigrations,0,3
6,Faroe Islands,Denmark,1393.000000,54738.0,83.09,58585.00000,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",Women,Immigrations,0,3
7,Faroe Islands,Denmark,1393.000000,54738.0,83.09,58585.00000,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",Women,Emigrations,0,3


In [5]:
ct_shpf_l0 = ct_shpf
ct_shpf_l0 = ct_shpf_l0.rename(columns={"2022": "val2022", "2023": "val2023"})

ct_shpf_long =  pd.wide_to_long(ct_shpf_l0, stubnames='val', i=['Country','Sex','Sta'], j='year')
# https://pandas.pydata.org/docs/reference/api/pandas.wide_to_long.html

In [6]:
ct_shpf_long

lifeExp  \
Country       Sex   Sta          year            
Denmark       Men   Immigrations 2022    80.70   
                                 2023    80.70   
                    Emigrations  2022    80.70   
                                 2023    80.70   
              Women Immigrations 2022    80.70   
                                 2023    80.70   
                    Emigrations  2022    80.70   
                                 2023    80.70   
Faroe Islands Men   Immigrations 2022    83.09   
                                 2023    83.09   
                    Emigrations  2022    83.09   
                                 2023    83.09   
              Women Immigrations 2022    83.09   
                                 2023    83.09   
                    Emigrations  2022    83.09   
                                 2023    83.09   

                                                                                geometry  \
Country       Sex   Sta          year                                                      
Denmark       Men   Immigrations 2022  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
                                 2023  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
                    Emigrations  2022  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
                                 2023  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
              Women Immigrations 2022  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
                                 2023  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
                    Emigrations  2022  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
                                 2023  MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...   
Faroe Islands Men   Immigrations 2022  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
                                 2023  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
                    Emigrations  2022  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
                                 2023  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
              Women Immigrations 2022  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
                                 2023  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
                    Emigrations  2022  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   
                                 2023  MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...   

                                           area_km2        pop    gdpPercap  \
Country       Sex   Sta          year                                         
Denmark       Men   Immigrations 2022  42759.927703  5643475.0  45057.07417   
                                 2023  42759.927703  5643475.0  45057.07417   
                    Emigrations  2022  42759.927703  5643475.0  45057.07417   
                                 2023  42759.927703  5643475.0  45057.07417   
              Women Immigrations 2022  42759.927703  5643475.0  45057.07417   
                                 2023  42759.927703  5643475.0  45057.07417   
                    Emigrations  2022  42759.927703  5643475.0  45057.07417   
                                 2023  42759.927703  5643475.0  45057.07417   
Faroe Islands Men   Immigrations 2022   1393.000000    54738.0  58585.00000   
                                 2023   1393.000000    54738.0  58585.00000   
                    Emigrations  2022   1393.000000    54738.0  58585.00000   
                                 2023   1393.000000    54738.0  58585.00000   
              Women Immigrations 2022   1393.000000    54738.0  58585.00000   
                                 2023   1393.000000    54738.0  58585.00000   
                    Emigrations  2022   1393.000000    54738.0  58585.00000   
                                 2023   1393.000000    54738.0  58585.00000   

                                      continent  val  
Cou

In [7]:
ct_shpf_longsb = ct_shpf_long
ct_shpf_longsb.reset_index(inplace=True)
#ct_shpf_longsb[['level_0','index']]
ct_shpf_longsb

,Country,Sex,Sta,year,lifeExp,geometry,area_km2,pop,gdpPercap,continent,val
0,Denmark,Men,Immigrations,2022,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,0
1,Denmark,Men,Immigrations,2023,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,3
2,Denmark,Men,Emigrations,2022,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,0
3,Denmark,Men,Emigrations,2023,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,3
4,Denmark,Women,Immigrations,2022,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,0
5,Denmark,Women,Immigrations,2023,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,3
6,Denmark,Women,Emigrations,2022,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,0
7,Denmark,Women,Emigrations,2023,80.70,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",42759.927703,5643475.0,45057.07417,Denmark,3
8,Faroe Islands,Men,Immigrations,2022,83.09,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",1393.000000,54738.0,58585.00000,Denmark,1
9,Faroe Islands,Men,Immigrations,2023,83.09,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",1393.000000,54738.0,58585.00000,Denmark,4


In [8]:
ct_shpf_df = pd.DataFrame(ct_shpf_longsb)
ct_shpf_df.drop("geometry",axis=1)
ct_shpf_geo = ct_shp[['Country','geometry']]

In [43]:

j = (ct_shpf_df.groupby(['Country','continent','area_km2','pop','lifeExp','gdpPercap'])
       .apply(lambda x: x[['Sex','Sta','year','val']].to_dict('records'))
       .reset_index()
       .rename(columns={0:'data'})
#       .to_json(orient='records')
 )

# https://stackoverflow.com/questions/40470954/convert-pandas-dataframe-to-nested-json

In [9]:
j = (ct_shpf_df.groupby(['Country','continent','area_km2','pop','lifeExp','gdpPercap'])
       .apply(lambda x: x.groupby('year').apply(lambda y: y[['Sex','Sta','val']].to_dict('records')).to_dict())
       .reset_index()
       .rename(columns={0:'data'})
#       .to_json(orient='records')
)

In [44]:
j.to_clipboard()

In [10]:
ct_shpf_geof  = ct_shpf_geo.merge(j, on='Country', how='left')


In [11]:
ct_shpf_geof

,Country,geometry,continent,area_km2,pop,lifeExp,gdpPercap,data
0,Denmark,"MULTIPOLYGON (((9.92191 54.98310, 9.28205 54.8...",Denmark,42759.927703,5643475.0,80.70,45057.07417,"{2022: [{'Sex': 'Men', 'Sta': 'Immigrations', ..."
1,Faroe Islands,"MULTIPOLYGON (((-6.71094 61.63698, -6.70991 61...",Denmark,1393.000000,54738.0,83.09,58585.00000,"{2022: [{'Sex': 'Men', 'Sta': 'Immigrations', ..."


In [12]:
#ct_shpf_geof.to_file('dataframe2.geojson', driver='GeoJSON') ValueError: Invalid field type <class 'list'>
#https://stackoverflow.com/questions/70777743/convert-geopandas-geodataframe-to-geojson
with open('dataframe_test5new.geojson' , 'w') as file:
    file.write(ct_shpf_geof.to_json())

this is a ref from R, but did not get time to test if that worked for SF....

In [47]:
STOP!

SyntaxError: invalid syntax (2635050600.py, line 1)

If create wide to long, try this's code


# ERROR


In [ ]:
df['geometry'] = df['geometry'].apply(wkt.loads)

# Now convert the pandas DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the coordinate reference system (CRS) if known
gdf.set_crs(epsg=4326, inplace=True) 

In [143]:
# geojson
def create_nested_structure(row):
    properties = row.drop('geometry').to_dict()  # Convert non-geometry columns to a dictionary
    properties['children'] = {'Sex': properties.pop('Sex'), 'Sta': properties.pop('Sta'),'Year': properties.pop('yer')}
    feature = {'type': 'Feature',
               'properties': properties,
               'geometry': json.loads(row['geometry'].to_json())}  # Convert geometry to GeoJSON
    return feature

# Apply the function to each row to create a list of GeoJSON features
features = ct_shpf_long.apply(create_nested_structure, axis=1).tolist()

# Create the final GeoJSON structure
geojson_structure = {'type': 'FeatureCollection', 'features': features}

# Convert the GeoJSON structure to a string
geojson_str = json.dumps(geojson_structure, indent=4)

# Save to a GeoJSON file
output_filename = 'output0.geojson'
with open(output_filename, 'w') as file:
    file.write(geojson_str)

KeyError: 'Sex'

In [113]:
ct_shpf.to_file('dataframe.geojson', driver='GeoJSON')

c:\Users\79038\anaconda3\envs\MUSA550\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [102]:
import networkx as nx

In [ ]:
g = nx.from_pandas_edgelist(ct_shpf, 'parent_id', 'child_id', create_using=nx.DiGraph())

df1 = df.rename(lambda x: '_'.join(x.split('_')[::-1]), axis=1)
df1 = pd.wide_to_long(df1.reset_index(), stubnames=['id', 'name'],
                i='index', j='type', suffix='\w+', sep='_')
d = {v: {'level': l, 'name': n} for v,l,n in zip(df1.id, df1.level, df1.name)}
root_id = df.loc[0, 'parent_id']
d[root_id]['level'] = 0

nx.set_node_attributes(g, d)
out = [tree_data_custom(g, root_id)] 

In [125]:
data = {
    'tam': ['A', 'A', 'B', 'B', 'C', 'C'],
    'dram': ['cor', 'dis', 'cor', 'dis', 'cor', 'dis'],
    'green': ['yes', 'no', 'yes', 'no', 'yes', 'no']
}
df = pd.DataFrame(data)
df

,tam,dram,green
0,A,cor,yes
1,A,dis,no
2,B,cor,yes
3,B,dis,no
4,C,cor,yes
5,C,dis,no


In [126]:
import json

# Assuming you have a DataFrame 'df' with columns 'tam', 'dram', and 'green'
# df = pd.DataFrame(...)

# Function to create a nested dictionary with 'children'
def nest_children(row):
    return {
        'tam': row['tam'],
        'children': {
            'dram': row['dram'],
            'green': row['green']
        }
    }

grouped = df.groupby('tam').apply(lambda x: x[['dram', 'green']].to_dict('records')).reset_index(name='children')

# Convert each row of the grouped DataFrame to the desired dictionary structure
nested_data = grouped.apply(lambda row: {'tam': row['tam'], 'children': row['children']}, axis=1).tolist()

# Convert the list of dictionaries to a JSON string
json_data = json.dumps(nested_data, indent=4)

In [ ]:
import json

# Assuming you have a DataFrame 'df' with columns 'tam', 'dram', and 'green'
# df = pd.DataFrame(...)

# Function to create a nested dictionary with 'children'
def nest_children(row):
    return {
        'tam': row['tam'],
        'tam': row['tam'],
        'tam': row['tam'],
        'tam': row['tam'],
        'tam': row['tam'],
        'tam': row['tam'],
        'tam': row['tam'],
        
        'children': {
            'dram': row['dram'],
            'green': row['green']
        }
    }

grouped = df.groupby('tam').apply(lambda x: x[['dram', 'green']].to_dict('records')).reset_index(name='children')

# Convert each row of the grouped DataFrame to the desired dictionary structure
nested_data = grouped.apply(lambda row: {'tam': row['tam'], 'children': row['children']}, axis=1).tolist()

# Convert the list of dictionaries to a JSON string
json_data = json.dumps(nested_data, indent=4)

In [127]:
json_data

'[\n    {\n        "tam": "A",\n        "children": [\n            {\n                "dram": "cor",\n                "green": "yes"\n            },\n            {\n                "dram": "dis",\n                "green": "no"\n            }\n        ]\n    },\n    {\n        "tam": "B",\n        "children": [\n            {\n                "dram": "cor",\n                "green": "yes"\n            },\n            {\n                "dram": "dis",\n                "green": "no"\n            }\n        ]\n    },\n    {\n        "tam": "C",\n        "children": [\n            {\n                "dram": "cor",\n                "green": "yes"\n            },\n            {\n                "dram": "dis",\n                "green": "no"\n            }\n        ]\n    }\n]'

In [124]:
json_data

'[\n    {\n        "tam": "A",\n        "children": {\n            "dram": "cor",\n            "green": "yes"\n        }\n    },\n    {\n        "tam": "A",\n        "children": {\n            "dram": "dis",\n            "green": "no"\n        }\n    },\n    {\n        "tam": "B",\n        "children": {\n            "dram": "cor",\n            "green": "yes"\n        }\n    },\n    {\n        "tam": "B",\n        "children": {\n            "dram": "dis",\n            "green": "no"\n        }\n    },\n    {\n        "tam": "C",\n        "children": {\n            "dram": "cor",\n            "green": "yes"\n        }\n    },\n    {\n        "tam": "C",\n        "children": {\n            "dram": "dis",\n            "green": "no"\n        }\n    }\n]'

In [123]:
nested_data

[{'tam': 'A', 'children': {'dram': 'cor', 'green': 'yes'}},
 {'tam': 'A', 'children': {'dram': 'dis', 'green': 'no'}},
 {'tam': 'B', 'children': {'dram': 'cor', 'green': 'yes'}},
 {'tam': 'B', 'children': {'dram': 'dis', 'green': 'no'}},
 {'tam': 'C', 'children': {'dram': 'cor', 'green': 'yes'}},
 {'tam': 'C', 'children': {'dram': 'dis', 'green': 'no'}}]

In [ ]:
grouped = df.groupby('tam').apply(lambda x: x[['dram', 'green']].to_dict('records')).reset_index(name='children')

In [130]:
from IPython.display import FileLink

# Save the JSON data to a file as before
filename = './daaata.json'  # Specify the path
with open(filename, 'w') as file:
    file.write(json_data)

# Create a download link for the file
FileLink('./daaata.json')

d:\OneDrive - PennO365\penn\5th\MUSA611\dashboard\daaata.json